In [1]:
import cv2
import numpy as np
import random
from os import listdir
from os.path import isfile, join

import math
import operator
from munkres import Munkres, DISALLOWED

img_path = './CS585-BatImages/CS585-BatImages/Gray'

files = sorted([join(img_path,f) for f in listdir(img_path) if isfile(join(img_path,f))])
print (f'Images = {len(files)}')

Images = 151


In [2]:
class TrackMe:
    def __init__(self,initialPosition,ID):
        self.ID=ID
        self.kalman = cv2.KalmanFilter(4,2,0)
        self.kalman.measurementMatrix = np.array([[1, 0, 0, 0],
                                     [0, 1, 0, 0]], np.float32)

        self.kalman.transitionMatrix = np.array([[1, 0, 1, 0],
                                    [0, 1, 0, 1],
                                    [0, 0, 1, 0],
                                    [0, 0, 0, 1]], np.float32)

        self.kalman.processNoiseCov = np.array([[1, 0, 0, 0],
                                   [0, 1, 0, 0],
                                   [0, 0, 1, 0],
                                   [0, 0, 0, 1]], np.float32) * 0.03
        self.initialPosition=initialPosition
        self.history=[initialPosition]
        self.historym=[initialPosition]
        self.color = list(np.random.random(size=3) * 256) 
        self.kalman.predict()
        self.kalman.correct(np.asarray((0,0), np.float32))

In [3]:
def detectCentroid(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur=cv2.blur(cv2.blur(gray, (9,9)),(4,4))
    thres_output = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY,51,-5)
    #thres_output = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY,31,-8)

    contours, hierarchy = cv2.findContours(thres_output, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    centroidList=[]
    for i in range(len(contours)):

        cnt=contours[i]
        M=cv2.moments(cnt)
        if M['m00']==0:
            continue
        else:
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            centroidList.append((cx,cy))
          
    return centroidList, thres_output


In [4]:
def firstFrameInitalize(kalmanObjs,centroidList,distThreshold,numTracks):
    copy1=centroidList.copy()
    rem_idx = []
    for i in range(len(centroidList)):
        for j in range(i+1,len(centroidList)):
            currDist=math.sqrt((centroidList[i][0] - centroidList[j][0])**2 
                                           + (centroidList[i][1] - centroidList[j][1])**2)
            if currDist<distThreshold:
                rem_idx.append(i)

    n = 0
    for (i,centroid) in enumerate(centroidList):
        if n < len(rem_idx) and rem_idx[n]==i:
            n += 1
            continue
        kalmanObjs.append(TrackMe(centroid,numTracks))
        numTracks+=1
    
    return kalmanObjs,numTracks


In [5]:
def pairDistanceCentroids(currentCentroidList, previousCentroidList):
    X=len(currentCentroidList)
    Y=len(previousCentroidList)

    n = max(X,Y)
    distanceMatrix = [[0.0] * n for i in range(n)]
    
    for i in range(X):
        for j in range(Y):
            distanceMatrix[i][j]=math.sqrt((currentCentroidList[i][0] - previousCentroidList[j][0])**2 
                                            + (currentCentroidList[i][1] - previousCentroidList[j][1])**2)
            # if distanceMatrix[i][j]>100.0:
            #     distanceMatrix[i][j]=DISALLOWED
            #distanceMatrix[j][i] = distanceMatrix[i][j]
        
    return distanceMatrix

In [6]:
def drawMe(img,historyList,color):
    if len(historyList)>3:
        
        cv2.circle(img,historyList[0],3,color,-1)
        prev=historyList[0]
        for i in range(2,len(historyList)):
            curr=historyList[i]
            cv2.line(img,prev,curr,color,1)
            prev=curr
    return img

In [7]:
kalmanObjList=[]
img_idx=0
numTracks=0

cv2.destroyAllWindows()
cv2.namedWindow('Bats',cv2.WINDOW_AUTOSIZE)

#ZZZ
track_img = None

def dist(new_coord,prev_coord):
    return math.sqrt((new_coord[0] - prev_coord[0])**2 
                      + (new_coord[1] - prev_coord[1])**2)

img = []
for i in range(len(files)):
    img.append(cv2.imread(files[i]))
    
while img_idx<len(files):
    curr=img[img_idx]
    
    measuredCentroids,thresh_output = detectCentroid(curr)
    
    # print(f'measured: ')
    # for i,m in enumerate(measuredCentroids):
    #     print (f'{i}: {m} ', end='')
    # print(f'')
    
    contour_output = curr[:,:,:].copy()
    
    show_centroids=0
    if show_centroids > 0:
        if img_idx == show_centroids:
            input("break here")
        track_img = np.zeros (curr.shape, curr.dtype)
        name=f'Bats{img_idx}'
        cv2.namedWindow(name, cv2.WINDOW_AUTOSIZE)
        for i,m in enumerate(measuredCentroids):
            curr = cv2.circle (curr, m, 4, (255,255,255), 2)
            curr = cv2.putText(curr, f'{i}', m, cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 1, cv2.LINE_AA)
        cv2.imshow(name,curr)
        cv2.waitKey(100)
        img_idx += 1
        continue
        
    
    if img_idx==0:
        kalmanObjList,numTracks=firstFrameInitalize(kalmanObjList,measuredCentroids,20,numTracks)
        
        track_img = np.zeros (curr.shape, curr.dtype)

#        for (i,track) in enumerate(kalmanObjList):
            #track.kalman.predict()  
            #track_img = cv2.circle (track_img, track.initialPosition, 4, track.color, 2)
            #track_img = cv2.putText(track_img, f'{track.ID}', track.initialPosition, cv2.FONT_HERSHEY_SIMPLEX, 1, track.color, 2, cv2.LINE_AA)
            #print (f' {i}: {track.initialPosition}')
#         print(kalmanObjList[0].history)

        contour_output = cv2.addWeighted (contour_output, 1, track_img, 1, 0)
     
    
    else:
        def helper(t):
            p = t.kalman.predict()
            return (int(p[0] + t.historym[-1][0] ), int(p[1] + t.historym[-1][1]))
        
        predictionCentroids = [helper(t) for t in kalmanObjList]
        
        costMatrix=pairDistanceCentroids(measuredCentroids,predictionCentroids)
        # print(f'Current centroids={len(measuredCentroids)} Prediction centroids={len(predictionCentroids)}')
        
        m = Munkres()
        indexes= m.compute(costMatrix)
                    
        del_track=[]
        new_track=[]
        for track, centroid in indexes:
            del_centroid = False
            add_track = False
            if track<len(measuredCentroids) and centroid<len(kalmanObjList):
                
                d = dist(measuredCentroids[track], kalmanObjList[centroid].history[-1])               
                if d < 125:
                    kalmanObjList[centroid].history.append(predictionCentroids[centroid])
                    kalmanObjList[centroid].historym.append(measuredCentroids[track])
                
                    relativePosition=np.asarray(tuple(map(operator.sub,measuredCentroids[track],predictionCentroids[centroid])),np.float32)
                    kalmanObjList[centroid].kalman.correct(relativePosition)
           
                    continue
                else:
                    del_centroid=True
                    add_track=True

            if del_centroid:
                del_track.append(centroid)
                
            if centroid>=len(kalmanObjList) or add_track:
                ntrack = TrackMe(measuredCentroids[track],numTracks)
                numTracks += 1
                new_track.append(ntrack)
                
        for d in sorted(del_track, reverse=True):
            del kalmanObjList[d]
        kalmanObjList.extend(new_track)
                    
        contour_output = cv2.addWeighted (contour_output, 1, track_img, 1, 0)
        
        for tracker in kalmanObjList:

            contour_output=drawMe(contour_output,tracker.history,tracker.color)

    cv2.imshow('Bats',contour_output)

    if cv2.waitKey(100)&0xFF == 27:
        break

    img_idx+=1
    
cv2.destroyAllWindows()
